### Get stock prices data from polygon API

In [1]:
#!pip install -r requirements.txt

In [2]:
from stockify import * 
import datetime as dt 
from polygon import RESTClient
from polygon.rest import models
from http import HTTPStatus
    
API_key = ""
client = RESTClient(API_key)

In [3]:
## Put the API key (if we do pay) into ENV_VARS
current_date = dt.datetime.now().date()
stocks_tickers = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=True)

stocks_requirments = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=False)

dfs = []

stocks_tickers=['ARR']
for st in stocks_tickers:
    
    #try:
        raw_response = get_p_process_by_ticker(API_key,
                                            client,
                                            st,
                                            get_daily_time_buckets(max_lookback_years=100,
                                                                    current_is_timestamp=False,
                                                                    frequency='daily',
                                                                    include_weekend=False),
                                            stock_prices_frequency= 'daily')
        
        df = get_jsonList_prices_into_dataframe(get_serialized_price_raw_into_json(raw_response,st),
                                    ['close'])
    # If exceed request per minute
    #except (BadResponse) as e:
    #    continue
    #    print(e)

        dfs.append(df)

if len(dfs)>0: 
    dfs = pd.concat(dfs)

In [4]:
dfs.converted_utc_timestamp.min()

'2020-12-28 05:00:00'

In [9]:
items = client.list_dividends(ticker='ARR',pay_date_gt='2022-08-01')
count=0
for item in items:
    count = count+1

BadResponse: {"status":"ERROR","request_id":"ad74ed1b1b60840ea0df4422dfaebd81","error":"You've exceeded the maximum requests per minute, please wait or upgrade your subscription to continue. https://polygon.io/pricing"}

In [ ]:
def get_serialized_dividends_raw_resp(self,
                            ticker:str):
    ''' 
    Serializes a row raw response, including 
    the specified stock ticker for dividends.
    '''
    
    import json
    
    return {"stock":ticker,
                "Dividend": {
                        "cash_amount": self.cash_amount,
                        "declaration_date": self.declaration_date,
                        "dividend_type": self.dividend_type,
                        "ex_dividend_date": self.ex_dividend_date,
                        "frequency": self.frequency,
                        "pay_date": self.pay_date,
                        "record_date": self.record_date,
                }
         }

In [ ]:
a = get_serialized_price_raw_into_json(items,'ARR')

BadResponse: {"status":"ERROR","request_id":"59104d367d5edb2c0927da9f9cd867d8","error":"You've exceeded the maximum requests per minute, please wait or upgrade your subscription to continue. https://polygon.io/pricing"}

In [6]:
a 

NameError: name 'a' is not defined

In [ ]:
def get_serialized_price_raw_into_json(raw_response:list,
                                ticker:str):
    ''' 
    Takes a dump of Agg objects list and returns each row as a json
    in a jsonList.
    '''
    import json
    
    jsonList = []
    
    for raw in raw_response:
        serie = get_serialized_dividends_raw_resp(raw,ticker)
        
        jsonDict = json.loads(json.dumps(serie))
        
        jsonList.append(jsonDict)
        
    return jsonList

In [ ]:
get_serialized_price_raw_into_json(items,'ARR')

[]

In [ ]:
for r in items:
    print(next(r))